In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'tax-index'
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
retriever = database.as_retriever(search_kwargs={'k':4})

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain


In [12]:
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from tqdm import tqdm


pinecone_api_key = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'tax-index'
index = pc.Index(index_name)

database = PineconeVectorStore(index, embedding)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

#✅ 배치 업서트
batch_size = 3  # 더 작게 할수록 안전
document_list = loader.load_and_split(text_splitter=text_splitter)
for i in tqdm(range(0, len(document_list), batch_size)):
    batch = document_list[i:i+batch_size]
    try:
        database.add_documents(batch)
    except Exception as e:
        print(f"Error at batch {i}: {e}")

100%|██████████| 75/75 [01:28<00:00,  1.18s/it]


In [8]:
from pinecone import Pinecone

api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

index = pc.Index("tax-index")

# 전체 벡터 삭제 (기본 네임스페이스일 경우)
index.delete(delete_all=True, namespace="")

{}

In [51]:
#query = '연봉 5천만원의 거주자의 소득세는?'
query = '연봉 5천만원의 직장인의 소득세는?'
#query = "소득세 가세기간에 대해 알려 주세요?"

In [ ]:
retriever = database.as_retriever(search_kwargs={'k':4})
retriever.invoke(query)

[Document(id='954f3752-3e3d-4362-9bef-721d0423ac6c', metadata={'source': './tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1

In [ ]:
# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
#retrieved_docs = database.similarity_search(query, k=2)

In [ ]:
#len(retrieved_docs)

2

In [ ]:
#retrieved_docs

[Document(id='954f3752-3e3d-4362-9bef-721d0423ac6c', metadata={'source': './tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1

In [42]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [43]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\dolsa\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
# 키워드 개선 전
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    #retriever=database.as_retriever(),
    #chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
# 키워드 개선 후
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,    
    chain_type_kwargs={"prompt": prompt}
)

#query : 직장인 -> 거주자 chain 추가


In [45]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [46]:
ai_message

{'query': '연봉 5천만원의 직장인의의 소득세는?',
 'result': '연봉 5천만원의 직장인의 소득세는 해당 금액에 대한 기본세율이 적용됩니다. 일반적으로 대한민국의 소득세는 누진세 구조로 되어 있으므로, 정확한 세금 금액은 해당 연봉에 대한 공제 항목 등을 고려해 산출해야 합니다. 일반적인 기본세율 구조는 별도의 소득 구간에 따라 적용되므로, 보다 정확한 계산이 필요합니다.'}

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
#tax_chain = {"query": dictionary_chain} | qa_chain

In [55]:
new_question = dictionary_chain.invoke({"question": query})

In [50]:
query

'연봉 5천만원의 직장인의의 소득세는?'

In [49]:
new_question

'연봉 5천만원의 거주자의 소득세는?'

In [52]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [53]:
ai_response = tax_chain.invoke({"question": query})

In [54]:
ai_response

{'query': '연봉 5천만원의 거주자의 소득세는?',
 'result': '연봉 5천만원의 거주자의 소득세는 624만원입니다. 이 금액은 과세표준 5천만원 이하 구간의 세율을 적용하여 계산된 것입니다.'}